In [49]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import json

import pandas as pd
import time
from sklearn.feature_selection import SelectKBest, chi2,SelectPercentile
import re
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from scipy import sparse
import lightgbm as lgb
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split 
from sklearn.utils import shuffle
import gc
from collections import Counter
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
from pylab import *

In [50]:
import json
import os
import time


import jieba
import numpy as np
import pandas as pd
from gensim import matutils
from gensim.models.keyedvectors import KeyedVectors
from sklearn.cluster import MiniBatchKMeans
from tqdm import tqdm

from utils import char_cleaner, char_list_cheaner
from w2v import build_model
import warnings
warnings.filterwarnings('ignore')

In [51]:
train_data = pd.read_table('./input/oppo_round1_train_20180929.txt',names= ['prefix','query_prediction','title','tag','label'],header= None, encoding='utf-8').astype(str)
val_data = pd.read_table('./input/oppo_round1_vali_20180929.txt',names = ['prefix','query_prediction','title','tag','label'],header = None, encoding='utf-8').astype(str)
test_data = pd.read_table('./input/oppo_round1_test_A_20180929.txt',names = ['prefix','query_prediction','title','tag'],header = None,encoding='utf-8').astype(str)
train_data = train_data[train_data['label'] != '音乐' ]
test_data['label'] = -1

In [52]:
import jieba

In [53]:
train_data = pd.concat([train_data,val_data])
train_data['label'] = train_data['label'].apply(lambda x: int(x))
test_data['label'] = test_data['label'].apply(lambda x: int(x))

In [54]:
len(train_data)

2049998

In [55]:
len(test_data)

50000

In [56]:
data = pd.concat([train_data, test_data])
data = data.fillna(-1)

In [57]:
del train_data
del test_data
gc.collect()

387

In [58]:
import os
import tqdm

def char_cleaner(char):
    if not isinstance(char, str):
        char = "null"

    pattern = re.compile("[^a-zA-Z\u4E00-\u9FA5 ]")
    char = re.sub(pattern, "", char)
    char = char.lower()
    return char


def char_list_cheaner(char_list, stop_words=None):
    new_char_list = list()
    for char in char_list:
        if len(char) <= 1:
            continue
        if stop_words and char in stop_words:
            continue
        new_char_list.append(char)

    return new_char_list

In [59]:
data["prefix"] = data["prefix"].apply(char_cleaner)
data["title"] = data["title"].apply(char_cleaner)

In [82]:
len(data[(data.label == 1)])/len(data[data.label != -1])

0.3720881678908955

In [83]:
import re

def remove_cha(x):
     
    x = re.sub('[\s+\.\!\/_,$%^*(+\"\')]+|[+——()?【】“”！，。？、~@#￥%……&*（）]+', "", str(x))
    x = x.replace('2C', '')
    return x

In [87]:
data.columns

Index(['prefix', 'query_prediction', 'title', 'tag', 'label', 'prefix_kmeans',
       'title_kmeans', 'is_in_title', 'leven_distance', 'distance_rate',
       'prefix_w2v'],
      dtype='object')

In [90]:
def is_prefix_in_title(prefix, title):
    #print(prefix)

    #print(title)
    return title.find('prefix')

In [91]:
data['is_prefix_in_title'] = data.apply(lambda row: is_prefix_in_title(row['prefix'], row['title']), axis = 1)

In [92]:
data['query_prediction_len'] = data['query_prediction'].apply(lambda x:str(x).count(',') + 1)

for i in range(1, max(data.query_prediction_len)):
    a = len(data[(data.label != -1)&(data.query_prediction_len == i)])
    b = len(data[(data.label == 1)&(data.query_prediction_len == i)])
    print(i, ':', a, b/a)

In [93]:
data.columns

Index(['prefix', 'query_prediction', 'title', 'tag', 'label', 'prefix_kmeans',
       'title_kmeans', 'is_in_title', 'leven_distance', 'distance_rate',
       'prefix_w2v', 'is_prefix_in_title', 'query_prediction_len'],
      dtype='object')

In [94]:
data['prefix_len'] = data.prefix.apply(lambda x:len(str(x)))

for i in range(0, max(data.prefix_len)):
    a = len(data[(data.label != -1)&(data.prefix_len == i)])
    b = len(data[(data.label == 1)&(data.prefix_len == i)])
    if b != 0:
        print(i, ':', a, b/a)

In [95]:
data['title_len'] = data.title.apply(lambda x:len(str(x)))

for i in range(0, max(data.title_len)):
    a = len(data[(data.label != -1)&(data.title_len == i)])
    b = len(data[(data.label == 1)&(data.title_len == i)])
    if b != 0:
        print(i, ':', a, b/a)

In [96]:
data['title-prefix_len'] = data.title_len - data.prefix_len

In [97]:

def get_max_query_prediction(x):
    if len(x) == 0:
        return 0
    x = x.values()
    return max(x)
data['query_prediction_max'] = data['query_prediction'].apply(lambda x:get_max_query_prediction(x))

In [98]:
data.query_prediction_max = data.query_prediction_max.astype(float)

i = 0.0
span = 0.1
while i <= 1:
    a = len(data[(data.query_prediction_max < (float(i)+span))&(data.label != -1)&(data.query_prediction_max >= float(i))])
    b = len(data[(data.query_prediction_max < (float(i)+span))&(data.label == 1)&(data.query_prediction_max >= float(i))])
    print('cll count:', a, ' ratio: ', b/a)
    i += span

In [99]:
def get_query_prediction_keys(x):
    
    x = x.keys()
    x = [remove_cha(value) for value in x]
    
    return ' '.join(x)

data['query_prediction_keys'] = data.query_prediction.apply(lambda x:get_query_prediction_keys(x))

In [100]:

def len_title_in_query(title, query):
    query = query.split(' ')
    if len(query) == 0:
        return 0
    l = 0
    for value in query:
        #print(value)
        if value.find(title) >= 0:
            #print(l)
            l += 1
    # print(l)
    return l

data['is_title_in_query_keys'] = data.apply(lambda row:len_title_in_query(row['title'], row['query_prediction_keys']),axis = 1)

for i in range(1, 11):
    a = len(data[(data.is_title_in_query_keys >= i)&(data.label == 1)])/len(data[(data.is_title_in_query_keys >= i)&(data.label != -1)])
    print(i, ':', a)

In [101]:
def get_query_predictions_values(x, index):
    
    length = len(x)
    if length <= index:
        return 0
    else:
        x = x.values()
        x = [float(value) for value in x]
        x = sorted(x)
        return x[index]
        
data['query_predictions_values_1'] = data.query_prediction.apply(lambda x:get_query_predictions_values(x, 1))

In [102]:
data['query_predictions_values_2'] = data.query_prediction.apply(lambda x:get_query_predictions_values(x, 2))
data['query_predictions_values_3'] = data.query_prediction.apply(lambda x:get_query_predictions_values(x, 3))
data['query_predictions_values_4'] = data.query_prediction.apply(lambda x:get_query_predictions_values(x, 4))
data['query_predictions_values_5'] = data.query_prediction.apply(lambda x:get_query_predictions_values(x, 5))
data['query_predictions_values_6'] = data.query_prediction.apply(lambda x:get_query_predictions_values(x, 6))
data['query_predictions_values_7'] = data.query_prediction.apply(lambda x:get_query_predictions_values(x, 7))
data['query_predictions_values_8'] = data.query_prediction.apply(lambda x:get_query_predictions_values(x, 8))
data['query_predictions_values_9'] = data.query_prediction.apply(lambda x:get_query_predictions_values(x, 9))

i = 0.0
span = 0.1
while i <= 1:
    a = len(data[(data.query_predictions_values_1 < (float(i)+span))&(data.label != -1)&(data.query_predictions_values_1 >= float(i))])
    b = len(data[(data.query_predictions_values_1 < (float(i)+span))&(data.label == 1)&(data.query_predictions_values_1 >= float(i))])
    if a != 0:
        print('cll count:', a, ' ratio: ', b/a) 
    i += span

In [103]:
def get_query_predictions_keys_len(x, index):
    length = len(x)
    x = x.keys()

In [104]:

len(data.columns)

28

In [105]:
data.columns

Index(['prefix', 'query_prediction', 'title', 'tag', 'label', 'prefix_kmeans',
       'title_kmeans', 'is_in_title', 'leven_distance', 'distance_rate',
       'prefix_w2v', 'is_prefix_in_title', 'query_prediction_len',
       'prefix_len', 'title_len', 'title-prefix_len', 'query_prediction_max',
       'query_prediction_keys', 'is_title_in_query_keys',
       'query_predictions_values_1', 'query_predictions_values_2',
       'query_predictions_values_3', 'query_predictions_values_4',
       'query_predictions_values_5', 'query_predictions_values_6',
       'query_predictions_values_7', 'query_predictions_values_8',
       'query_predictions_values_9'],
      dtype='object')

In [106]:
data['prefix_title_tag'] = data['prefix'].astype(str)+'_'+data['title'].astype(str)+'_'+data['tag'].astype(str)

click_list = []
for index, row in data[data.label != -1].iterrows():
    if row['label'] == 1:
        if row['prefix_title_tag'] not in click_list:
            click_list.append(row['prefix_title_tag'])
    if index % 100000 == 0:
        print(index)

In [107]:
click_list = np.load('cick_list.npy')

In [108]:
len(click_list)

166269

np.save('cick_list.npy', np.array(click_list))

In [109]:
# data['is_equal_title_prefix'] = data.apply(lambda row: int(row['title'] == row['prefix']),axis = 1)

In [110]:
a = data[data.label != -1].prefix_title_tag.value_counts().to_dict()
data['prefix_title_tag_len'] = data.prefix_title_tag.apply(lambda x:a[x] if x in a.keys() else 0)

len(data[data.prefix_title_tag_len <= 10])

def write_file_w2c(values):
    with open('w2c.txt', 'a') as file:
        for value in values:
            if isinstance(value, dict):
                for key in value.keys():
                    line = jieba.cut(key)
                    file.write(' '.join(line) + '\n')
            else:
                # print(value)
                line = jieba.cut(value)
                # print(' '.join(line))
                file.write(' '.join(line) + '\n')
                
write_file_w2c(list(data.prefix))
write_file_w2c(list(data.title))
write_file_w2c(list(data.query_prediction))

from gensim.models import word2vec
import logging
import time

logging.basicConfig(format='%(asctime)s:%(levelname)s: %(message)s', level=logging.INFO)
sentences =word2vec.Text8Corpus(u"w2c.txt")  # 加载语料
start = time.time()
model =word2vec.Word2Vec(sentences, size=500, min_count = 0)
print(time.time() - start)

def simi_prefix_title(prefix, title, model):

    # print(prefix)
    prefix = prefix.strip()
    title = title.strip()
    prefix = prefix.replace(' ', '')
    title = title.replace(' ', '')
    
    
    if prefix == '' or title == '' or prefix == ' ' or title == ' ':
        return 0
    
    prefix = jieba.cut(prefix)
    prefix = ' '.join(prefix)
    prefix = prefix.split(' ')
    
    
    title = jieba.cut(title)
    title = ' '.join(title)
    title = title.split(' ')
    
    print('prefix', (prefix))
    print('title', (title))
    
    
    print(model.similarity(prefix[0], title[0]))
    
    
    all_sum = []
    for i in range(len(prefix)):
        
        if prefix[i] == '' or prefix[i] == ' ':
            print('i is ', i)
            continue
        sum_title = []
        for j in range(len(title)):
            if title[j] == '' or title[j] == ' ':
                continue 
            else:
                sum_title.append(model.similarity(prefix[i], title[j]))
    all_sum.append(sum_title)
    print(all_sum)
    #prefix = jieba.cut(prefix)
    #prefix = ' '.join(prefix)
    #prefix = prefix.split(' ')
    #print(prefix)
    #title = jieba.cut(title)
    #title = ' '.join(title)
    #title = title.split(' ')
    #print(prefix)
    # return model.n_similarity(prefix, title)
    
    return model.n_similarity(prefix, title)
data['prefix_title_smilar'] = data.apply(lambda row : simi_prefix_title(row['prefix'].strip(), row['title'].strip(), model), axis = 1)

In [111]:
# is_click_history = np.load('is_click_history.npy')

In [112]:
# data['is_click_history'] = is_click_history 

np.save('is_click_history_filter_1.npy', np.array(data.is_click_history))

In [113]:
# data['is_click_history'] = np.load('is_click_history_filter_5.npy')

In [114]:
# data['is_click_history'] = np.load('is_click_history_filter_1.npy')

In [115]:
# len(data[(data.label == 1)&(data.is_click_history == 1)])/len(data[(data.label != -1)&(data.is_click_history == 1)])

is_click_history = []
for index, row in data.iterrows():
    if row['prefix_title_tag_len'] <= 1:
        is_click_history.append(-1)
    else:
        if row['prefix_title_tag'] in click_list:
            is_click_history.append(1)
        else:
            is_click_history.append(0)
    if index % 100000 == 0:
        print(index)
        
data['is_click_history'] = is_click_history        

In [116]:
# data['prefix_title_tag_history_click_len'] = prefix_title_tag_history_click_len

for i in range(0, 1000):
    a = len(data[(data.prefix_title_tag_history_click_len > i)&(data.label == 1)])
    b = len(data[(data.prefix_title_tag_history_click_len > i)&(data.label != -1)])
    print(i, ':', a, '  ', a/b)

In [117]:
# data['prefix_groupby_title_tag_len'] = np.load('./prefix_group_title_tag_len.npy')

data['prefix_title_tag'] = LabelEncoder().fit_transform(data['prefix_title_tag'])
a = data[data.label != -1].groupby(['prefix', 'prefix_title_tag'], as_index = False)['label'].agg({'count':'count', 'sum':'sum'})
a['ratio'] = a['sum']/a['count']
a['filter_5'] = a['count'].apply(lambda x:1 if x>2 else 0)
a['ratio'] = a['ratio']*a['filter_5']

max_prefix_title_tag = []
for index, prefix in enumerate(a.prefix.unique()):
    temp = a[a['prefix'] == prefix]
    # print(type(temp))
    # print(temp)
    if len(temp) != 1:
        max_ratio = max(temp.ratio)
        # print(max_ratio)
        # print(temp)
        if max_ratio != 0:
            # length = len(temp[temp.ratio == max_ratio])
            # for i in range(length):
            t = temp[temp.ratio == max_ratio]
            # print('t ; ---------')
            # print(t)
            t = list(t['prefix_title_tag'])
            # print(t)
            # print(t)
            for i in range(len(t)):
                max_prefix_title_tag.append(t[i])
    if index % 10000 == 0:
        print(index)

data['max_ratio_prefix_title_tag'] = data.prefix_title_tag.apply(lambda x:1 if x in max_prefix_title_tag else 0)

In [118]:
# np.save('max_ratio_prefix_title_tag_2.npy', np.array(data.max_ratio_prefix_title_tag))

In [119]:
# data['max_ratio_prefix_title_tag'] = np.load('max_ratio_prefix_title_tag.npy')

In [120]:
def _levenshtein_distance(prefix, title):
        str1 = prefix
        str2 = title

        if not isinstance(str1, str):
            str1 = "null"

        x_size = len(str1) + 1
        y_size = len(str2) + 1

        matrix = np.zeros((x_size, y_size), dtype=np.int_)

        for x in range(x_size):
            matrix[x, 0] = x

        for y in range(y_size):
            matrix[0, y] = y

        for x in range(1, x_size):
            for y in range(1, y_size):
                if str1[x - 1] == str2[y - 1]:
                    matrix[x, y] = min(matrix[x - 1, y] + 1, matrix[x - 1, y - 1], matrix[x, y - 1] + 1)
                else:
                    matrix[x, y] = min(matrix[x - 1, y] + 1, matrix[x - 1, y - 1] + 1, matrix[x, y - 1] + 1)

        return matrix[x_size - 1, y_size - 1]
    
# data['prefix_title_levenshtein_distance'] = data.apply(lambda row: _levenshtein_distance(row['prefix'], row['title']), axis = 1)

In [121]:
# data[['prefix', 'title', 'title-prefix_len', 'prefix_title_levenshtein_distance']]

In [122]:
# data['title-prefix_len_ratio'] = data['title-prefix_len']/data['title_len']

In [123]:
#data[['title_len', 'title-prefix_len', 'title-prefix_len_ratio']]

In [124]:
# def sum_query(values):
    
data['sum_query_prediction_value'] = data.query_prediction.apply(lambda x:sum([float(v) for v in x.values()]))

In [125]:
data[['sum_query_prediction_value', 'label']]

,sum_query_prediction_value,label
0,0.373,0
1,0.520,0
2,0.520,1
3,0.404,1
4,0.747,0
5,1.001,1
6,0.331,0
7,0.364,0
8,1.001,1
9,0.540,0


for i in range(200):
    a = len(data[(data.label == 1)&(data.sum_query_prediction_value >= i*0.01)])
    b = len(data[(data.label != -1)&(data.sum_query_prediction_value >= i*0.01)])
    print(i, ' ', b, ' ', a/b)

In [126]:
one_hot_feature = ['prefix', 'title', 'tag']

cate = [
        'query_prediction_len', 
    
        'prefix_len', 
        'title_len',   
        'title-prefix_len', 
    
        'query_prediction_max',
    
    
        'is_title_in_query_keys',
    
        'query_predictions_values_1',
        'query_predictions_values_2',
        'query_predictions_values_3', 
        'query_predictions_values_4',
        'query_predictions_values_5', 
         'query_predictions_values_6',
        'query_predictions_values_7', 
         'query_predictions_values_8',
       'query_predictions_values_9',
        
        'is_prefix_in_title',
    
       # 'prefix_title_tag_history_click_len',
    
        # 'is_click_history',
        'sum_query_prediction_value',
        #'is_equal_title_prefix',
        # 'prefix_groupby_title_tag_len',
    
        # 'max_ratio_prefix_title_tag',
        # 'title-prefix_len_ratio',
        #'click_sum',
    
    'prefix_kmeans',
       'title_kmeans',
     'prefix_w2v',
    
       ]

vector_feature = ['query_prediction_keys']

In [127]:
features = ['prefix', 
            'title', 
            'tag', 
            
            'is_prefix_in_title',
            
            'is_title_in_query_keys', 
            
            # 'is_click_history',
            
            #'query_prediction_len',
            
            # 'max_ratio_prefix_title_tag',
            
            'prefix_len', 
            'title_len',   
            'title-prefix_len',
            
            
               'prefix_kmeans',
               'title_kmeans',
           ]

# features.append('prefix_title_tag')

for feature in features:
    a = data[feature].value_counts().to_dict()
    data[feature + '_count'] = data[feature].apply(lambda x:a[x])
    cate.append(feature+'_count')
    
# del features[-1]

In [128]:
for index, i in enumerate(features[:-1]):
    for j in features[index+1:]:
        new_feature = i+'_'+j
        data[new_feature] = data[i].astype(str) + '_' + data[j].astype(str)
        data[new_feature] = LabelEncoder().fit_transform(data[new_feature])
        cate.append(new_feature)
        new_feature_count = new_feature + '_count'
        a = data[new_feature].value_counts().to_dict()
        data[new_feature_count] = data[new_feature].apply(lambda x:a[x])
        cate.append(new_feature_count)
        print(i, ':', j, ' finish!')
        # temp = data.groupby(new_feature, as_index=False)['label'].agg({new_feature+'_click': 'sum'})
        # temp['_'.join(item_g)+'_ctr'] = temp['_'.join(item_g)+'_click']/(temp['_'.join(item_g)+'count']+3)
        # data = pd.merge(data, temp, on=new_feature, how='left')
        # cate.append(new_feature+'_click')   

prefix : title  finish!
prefix : tag  finish!
prefix : is_prefix_in_title  finish!
prefix : is_title_in_query_keys  finish!
prefix : prefix_len  finish!
prefix : title_len  finish!
prefix : title-prefix_len  finish!
prefix : prefix_kmeans  finish!
prefix : title_kmeans  finish!
title : tag  finish!
title : is_prefix_in_title  finish!
title : is_title_in_query_keys  finish!
title : prefix_len  finish!
title : title_len  finish!
title : title-prefix_len  finish!
title : prefix_kmeans  finish!
title : title_kmeans  finish!
tag : is_prefix_in_title  finish!
tag : is_title_in_query_keys  finish!
tag : prefix_len  finish!
tag : title_len  finish!
tag : title-prefix_len  finish!
tag : prefix_kmeans  finish!
tag : title_kmeans  finish!
is_prefix_in_title : is_title_in_query_keys  finish!
is_prefix_in_title : prefix_len  finish!
is_prefix_in_title : title_len  finish!
is_prefix_in_title : title-prefix_len  finish!
is_prefix_in_title : prefix_kmeans  finish!
is_prefix_in_title : title_kmeans  fi

In [129]:

pos_features = ['query_prediction_len', 'prefix_len', 'title_len', 'title-prefix_len']
for feature in pos_features:
    temp = data.groupby(feature, as_index=False)['label'].agg({feature+'_click_': 'sum'})
    # temp['_'.join(item_g)+'_ctr'] = temp['_'.join(item_g)+'_click']/(temp['_'.join(item_g)+'count']+3)
    data = pd.merge(data, temp, on=feature, how='left')
    cate.append(feature+'_click_')
    print(feature)


query_prediction_len
prefix_len
title_len
title-prefix_len


In [136]:
cate

['query_prediction_len',
 'prefix_len',
 'title_len',
 'title-prefix_len',
 'query_prediction_max',
 'is_title_in_query_keys',
 'query_predictions_values_1',
 'query_predictions_values_2',
 'query_predictions_values_3',
 'query_predictions_values_4',
 'query_predictions_values_5',
 'query_predictions_values_6',
 'query_predictions_values_7',
 'query_predictions_values_8',
 'query_predictions_values_9',
 'is_prefix_in_title',
 'sum_query_prediction_value',
 'prefix_kmeans',
 'title_kmeans',
 'prefix_w2v',
 'prefix_count',
 'title_count',
 'tag_count',
 'is_prefix_in_title_count',
 'is_title_in_query_keys_count',
 'prefix_len_count',
 'title_len_count',
 'title-prefix_len_count',
 'prefix_kmeans_count',
 'title_kmeans_count',
 'prefix_title',
 'prefix_title_count',
 'prefix_tag',
 'prefix_tag_count',
 'prefix_is_prefix_in_title',
 'prefix_is_prefix_in_title_count',
 'prefix_is_title_in_query_keys',
 'prefix_is_title_in_query_keys_count',
 'prefix_prefix_len',
 'prefix_prefix_len_count',


In [141]:
cate

['query_prediction_len',
 'prefix_len',
 'title_len',
 'title-prefix_len',
 'query_prediction_max',
 'is_title_in_query_keys',
 'query_predictions_values_1',
 'query_predictions_values_2',
 'query_predictions_values_3',
 'query_predictions_values_4',
 'query_predictions_values_5',
 'query_predictions_values_6',
 'query_predictions_values_7',
 'query_predictions_values_8',
 'query_predictions_values_9',
 'is_prefix_in_title',
 'sum_query_prediction_value',
 'prefix_kmeans',
 'title_kmeans',
 'prefix_w2v',
 'prefix_count',
 'title_count',
 'tag_count',
 'is_prefix_in_title_count',
 'is_title_in_query_keys_count',
 'prefix_len_count',
 'title_len_count',
 'title-prefix_len_count',
 'prefix_kmeans_count',
 'title_kmeans_count',
 'prefix_title',
 'prefix_title_count',
 'prefix_tag',
 'prefix_tag_count',
 'prefix_is_prefix_in_title',
 'prefix_is_prefix_in_title_count',
 'prefix_is_title_in_query_keys',
 'prefix_is_title_in_query_keys_count',
 'prefix_prefix_len',
 'prefix_prefix_len_count',


In [142]:
def find_best_threshold(true_y, train, clf):
    predict = clf.predict_proba(train, num_iteration=clf.best_iteration_)[:, 1]
    # print(predict)
    span = list(np.linspace(0.3,0.50, 1000))
    scores = []
    for i in span:
        # print(i)
        predict_y =  np.where(predict >= i, 1,0)
    # print(predict_y)
        score = f1_score(true_y, predict_y)
        # print(score)
        scores.append(score)
        
    # print(scores)
    best_score = max(scores)
    best_index = scores.index(best_score)
    print(best_score)
    return span[best_index], best_score
    # threshold = [i for i in range(0.1, 0.8, 0.01)]
        

In [143]:
data = data.reset_index(drop = True)

data.to_csv('data.csv', index = False)

In [144]:
f1_best_scores = []

np.save('max_ratio_prefix_title_tag.npy', np.array(data.max_ratio_prefix_title_tag))

In [145]:
for feature in one_hot_feature:
    # print(feature)
    try:
        data[feature] = LabelEncoder().fit_transform(data[feature].apply(int))
    except:
        data[feature] = LabelEncoder().fit_transform(data[feature])

train=data[data.label!=-1]

# train = train.sample(frac = 1).reset_index(drop = True)

train_y = train.pop('label')

test = data[(data.label == -1)]

test = test.drop('label',axis=1)
res = test[['prefix']]

train_x = train[cate + one_hot_feature]

test_x = test[cate + one_hot_feature]

'''
for feature in one_hot_feature:

    if len(data[feature].unique()) > 5:
        enc = OneHotEncoder()
        enc.fit(data[feature].values.reshape(-1, 1))

        # train_a = enc.transform(train[feature].values.reshape(-1, 1))
        test_a = enc.transform(test[feature].values.reshape(-1, 1))

        # print(train_a.shape)
        # feature_select = SelectPercentile(chi2, percentile=95)
        # feature_select.fit(train_a, train_y)

        # train_x= sparse.hstack((train_x, train_a))
        test_x = sparse.hstack((test_x, test_a))

    print(feature+' finish')
print('one-hot prepared !')
'''

# train_x = np.array(train_x)
# train_y = np.array(train_y)
# test_x = np.array(test_x)

print("LGB test")

clf = lgb.LGBMClassifier(
    boosting_type='gbdt', subsample=0.7, colsample_bytree=0.7, 
    max_depth=-1, n_estimators=10000, objective='binary',min_child_weight = 5, 
    subsample_freq=1, num_leaves=127, reg_alpha=0,reg_lambda = 1,
     random_state=2018, n_jobs=-1, learning_rate=0.05
)

skf = StratifiedKFold(n_splits=5, random_state=2018, shuffle=True)
best_scores = []


LGB test


In [146]:
for index, (train_index, test_index) in enumerate(skf.split(train_x, train_y)):
    print(type(train_index))

    x, y = train_x.iloc[train_index], train_y.iloc[train_index]
    valid_x, valid_y = train_x.iloc[test_index], train_y.iloc[test_index]
    
    '''
    x['prefix_title_tag'] = x['prefix'].astype(str) + x['title'].astype(str)+ x['tag'].astype(str)
    a = x['prefix_title_tag'].value_counts().to_dict()
    x['prefix_title_tag_len'] = x['prefix_title_tag'].apply(lambda x:a[x])
    
    valid_x['prefix_title_tag'] = valid_x['prefix'].astype(str)+valid_x['title'].astype(str)+valid_x['tag'].astype(str)
    # a = valid_x['prefix_title_tag'].value_counts().to_dict()
    valid_x['prefix_title_tag_len'] = valid_x['prefix_title_tag'].apply(lambda x:a[x] if x in a.keys() else 0)   
    
    x['label'] = y
    temp = x.groupby('prefix_title_tag')['label'].agg({'click_count':'count', 'click_sum':'sum'})
    temp['click_ratio'] = temp['click_sum']/temp['click_count']
    temp['click_ratio'] = round(temp['click_ratio']*10)
    del temp['click_count']
    del temp['click_sum']
    x = pd.merge(x, temp, on = 'prefix_title_tag', how= 'left')
    
    
    
    x['prefix_title_tag_len_filter_5'] = x.prefix_title_tag_len.apply(lambda x:1 if x >20 else 0)
    x['click_ratio'] = x['click_ratio']*x['prefix_title_tag_len_filter_5']
    
    del x['label']
    del x['prefix_title_tag']
    del x['prefix_title_tag_len']
    del x['prefix_title_tag_len_filter_5']
    
    valid_x = pd.merge(valid_x, temp, on = 'prefix_title_tag', how = 'left')
    valid_x = valid_x.fillna(0)
    
    
    del valid_x['prefix_title_tag']
    del valid_x['prefix_title_tag_len']
    
    '''
    
    '''
    x = x.reset_index(drop = True)
    
    final_x = x[cate]
    final_valid_x = valid_x[cate]
    
    for feature in one_hot_feature:

        if len(data[feature].unique()) > 5:
            enc = OneHotEncoder()
            enc.fit(data[feature].values.reshape(-1, 1))

            train_a = enc.transform(x[feature].values.reshape(-1, 1))
            valid_a = enc.transform(valid_x[feature].values.reshape(-1, 1))
       
            final_x= sparse.hstack((final_x, train_a))
            final_valid_x= sparse.hstack((final_valid_x, valid_a))
    
    '''
    '''
    row_y = list(y)
    
    x['prefix_title_tag'] = x['prefix'].astype(str) + x['title'].astype(str)+ x['tag'].astype(str)
    valid_x['prefix_title_tag'] = valid_x['prefix'].astype(str)+valid_x['title'].astype(str)+valid_x['tag'].astype(str)
    
    click_prefix_title_tag = dict()
    prefix_title_tag_history_click_len = []
    i = 0
    for index, row in x.iterrows():
        # print(index)
        if row_y[i] == 1:
            if row['prefix_title_tag'] in click_prefix_title_tag.keys():
                l = click_prefix_title_tag[row['prefix_title_tag']]
                click_prefix_title_tag[row['prefix_title_tag']] += 1
                prefix_title_tag_history_click_len.append(l)
            else:
                click_prefix_title_tag[row['prefix_title_tag']] = 1
                prefix_title_tag_history_click_len.append(0)
        else:
            if row['prefix_title_tag'] in click_prefix_title_tag.keys():
                prefix_title_tag_history_click_len.append(click_prefix_title_tag[row['prefix_title_tag']])
            else:
                prefix_title_tag_history_click_len.append(0)
        i += 1
    x['prefix_title_tag_history_click_len'] = prefix_title_tag_history_click_len
    valid_x['prefix_title_tag_history_click_len'] = valid_x['prefix_title_tag'].apply(lambda x:click_prefix_title_tag[x] if x in click_prefix_title_tag.keys() else 0)
    
    del x['prefix_title_tag']
    del valid_x['prefix_title_tag']
    '''
    
    print('fit start!')
    
    clf.fit(x, y,
              eval_set=[(x, y),
                        (valid_x, valid_y)], eval_metric='logloss',verbose = 0, early_stopping_rounds=100)
    best_scores.append(clf.best_score_['valid_1']['binary_logloss'])
    print(best_scores)
    best_threshold, best_score = find_best_threshold(valid_y, valid_x, clf)
    # f1 = f1_score(, np.where(clf.predict(,num_iteration=clf.best_iteration_)>=0.5, 1,0))

    f1_best_scores.append(best_score)
    test_pred = np.where(clf.predict_proba(test_x, num_iteration=clf.best_iteration_)[:, 1] >= best_threshold, 1, 0)
    res['predicted_score' + str(index)] = test_pred

    # res['predicted_score'] = res['predicted_score'].apply(lambda x: float('%.6f' % x))
    gc.collect()
    res=res.reset_index(drop=True)

print(' predict finish!')

<class 'numpy.ndarray'>
fit start!
[0.44097728679216425]
0.7304985090531787
<class 'numpy.ndarray'>
fit start!
[0.44097728679216425, 0.4410842314221583]
0.7306540427902625
<class 'numpy.ndarray'>
fit start!
[0.44097728679216425, 0.4410842314221583, 0.43894836756341604]
0.7326478543730753
<class 'numpy.ndarray'>
fit start!
[0.44097728679216425, 0.4410842314221583, 0.43894836756341604, 0.4412297471628775]
0.731117694652872
<class 'numpy.ndarray'>
fit start!
[0.44097728679216425, 0.4410842314221583, 0.43894836756341604, 0.4412297471628775, 0.43957613655833133]
0.731554826037642
 predict finish!


In [147]:
f1_best_scores

[0.7304985090531787,
 0.7306540427902625,
 0.7326478543730753,
 0.731117694652872,
 0.731554826037642]

In [148]:
sum(f1_best_scores)/len(f1_best_scores)
# 0.7297225014710185 only count
# 0.7292830874172596 add query_prediction_values 0.44938
# 0.732607113523913 add  cross and count and query_prediction_values
# 0.7306544906098257 add base  cross and count

0.731294585381406

In [149]:
predict = res

In [150]:
predict['predicted_score'] = predict.predicted_score0 + predict.predicted_score1 + predict.predicted_score2 + predict.predicted_score3 + predict.predicted_score4

predict['predicted_score'] += predict.predicted_score5 + predict.predicted_score6 + predict.predicted_score7 + predict.predicted_score8 + predict.predicted_score9

In [151]:
# predict.score = predict.predicted_score / 5

In [152]:
len(predict[predict.predicted_score >= 3])/len(predict)

0.44488

In [159]:
len(predict[predict.predicted_score >= 5])/len(predict)

0.35628

In [160]:
predict['score'] = predict.predicted_score >= 5

In [161]:
predict.score = predict.score.astype(int)

In [162]:
predict

,prefix,predicted_score0,predicted_score1,predicted_score2,predicted_score3,predicted_score4,predicted_score,score
0,134637,0,0,0,0,0,0,0
1,11982,0,1,0,1,1,3,0
2,62964,1,0,1,0,1,3,0
3,158759,0,0,0,0,0,0,0
4,38380,0,0,0,0,0,0,0
5,81633,0,0,0,0,0,0,0
6,19179,1,1,1,1,1,5,1
7,20310,0,0,0,0,0,0,0
8,65635,1,1,1,1,1,5,1
9,92184,0,0,0,0,0,0,0


In [163]:
predict.score.to_csv('submit.csv', index = False)

In [164]:
len(predict[predict.score == 1])/len(predict)

0.35628

In [ ]:
0.42612

In [ ]:
a = pd.read_csv('../submit.csv', header = None)
a.columns = ['b']

In [ ]:
len(a[a['b'] == 1])/len(a)

In [ ]:
b = pd.read_csv('./add_click.csv', header = None)
b.columns = ['b']

In [ ]:
len(b[b['b'] == 1])/len(b)

In [ ]:
a['a'] = b['b']

In [ ]:
b = pd.read_csv('./submit.csv', header = None)
b.columns = ['b']

In [ ]:
len(b[b['b'] == 1])/len(b)

In [ ]:
a['c'] = b['b']

In [ ]:
len(predict)